In [107]:
# Création et paramétrage du port série
import serial
if 'ser' in locals(): ser.close()
ser = serial.Serial()
ser.baudrate = 9600 
ser.port = 'COM5' # 'COMx' sous windows Ou '/dev/tty.usbmodem1411' par exemple, sous MacOS et Linux
ser.open()
ser.is_open

True

In [108]:
n = 0
while(n < 10):
    valueString = ser.readline()
    value = int(valueString.strip())
    value = 5.0/1024.0*value
    value = (value - 0.75)/0.01 + 25
    print(value)
    n = n + 1 
ser.close()

26.66015625
27.1484375
27.1484375
27.1484375
27.1484375
27.1484375
27.1484375
27.1484375
27.1484375
26.66015625
